In [172]:
import pandas as pd
import requests
import faiss
import re
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
import pandas as pd
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\siddh\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

INPUT FROM API

In [173]:
search = input('What are you looking for?\n')

NEWS_API_KEY = 'd6c7f3bd5dea44c2bcca2879c219183b'
NEWS_API_ENDPOINT = 'https://newsapi.org/v2/everything'

def fetch_news_for_title(title, num_articles=100):
    params = {
        'q': title,
        'language': 'en',
        'apiKey': NEWS_API_KEY,
        'pageSize': num_articles
    }
    response = requests.get(NEWS_API_ENDPOINT, params=params)
    
    if response.status_code == 200:
        news_data = response.json()
        articles = news_data['articles']
        
        # Create a DataFrame to store the news data
        df = pd.DataFrame(articles)
        
        return df
    else:
        print("Failed to fetch news:", response.status_code)
        return None

# Example: Fetch news articles for the title "technology"
news_df = fetch_news_for_title(search)

# Display the DataFrame
#print(news_df)


PREPROCESS API DATA

In [174]:
news_df.rename(columns = {'title':'headline', 'description' : 'abstract', 'url':'links'}, inplace = True) 
news_df = news_df.drop(columns=['source', 'urlToImage', 'publishedAt'])
news_df.head()

,author,headline,abstract,links,content
0,None,The mind behind the 'doped Olympics',"The Enhanced Games has been derided as ""a clow...",https://www.bbc.co.uk/sport/68672104,Aron D'Souza at the Enhanced Games headquarter...
1,None,'It feels right' - Whitlock to retire after Ol...,Gymnast Max Whitlock tells BBC Breakfast he wi...,https://www.bbc.co.uk/sport/av/gymnastics/6877...,'It feels right' - Whitlock to retire after Ol...
2,BBC Sport,'Paris will be my final Olympics' - GB's Whitlock,Britain's most successful gymnast Max Whitlock...,https://www.bbc.co.uk/sport/gymnastics/68768442,Three-time gold medallist Max Whitlock says Pa...
3,https://www.facebook.com/bbcnews,Macron says Paris Olympics opening could be moved,The French president says the Olympics opening...,https://www.bbc.co.uk/news/world-europe-68815081,French President Emmanuel Macron says the Pari...
4,Matthew Henry,'Like an Olympic final' - Richards secures Par...,Current and former world champions Freya Colbe...,https://www.bbc.co.uk/sport/swimming/68736943,Current and former world champions Freya Colbe...


In [175]:
news_df.sample(5)

,author,headline,abstract,links,content
99,peterdeadline,‘Call Her Daddy’s Alex Cooper Heading To Paris...,Peacock is adding some A-list podcaster firepo...,http://deadline.com/2024/04/nbc-olympics-watch...,Peacock is adding some A-list podcaster firepo...
10,Emma Bowman,"With 100 days before the Paris Olympics, Team ...",U.S. Olympic and Paralympic Committee official...,https://www.npr.org/2024/04/17/1245221624/100-...,Boxer Morelle McCane speaks to reporters at th...
75,Suzanne Nuyen,Iran hails Israel strikes as a 'victory'; Phot...,Iran says its attack against Israel was a succ...,https://www.npr.org/2024/04/16/1245005448/up-f...,Good morning. You're reading the Up First news...
64,None,Peaty seals Olympics spot with world leading time,Watch as Adam Peaty seals his Paris Olympics s...,https://www.bbc.co.uk/sport/av/swimming/68719174,Watch as Adam Peaty seals his Paris Olympics s...
88,None,WBD plans to launch Max in Europe starting on ...,Adam Chitwood / The Wrap:\nWBD plans to launch...,https://www.techmeme.com/240321/p37,About This Page\r\nThis is a Techmeme archive ...


In [176]:
metadata = news_df

metadata['text'] = metadata['headline'] + " " + metadata['abstract']
metadata['text'] = metadata['text'].apply(lambda x: x.lower())
metadata['text'] = metadata['text'].apply(lambda x: re.sub(r'\W+', ' ', x))

In [177]:
metadata.head()

,author,headline,abstract,links,content,text
0,None,The mind behind the 'doped Olympics',"The Enhanced Games has been derided as ""a clow...",https://www.bbc.co.uk/sport/68672104,Aron D'Souza at the Enhanced Games headquarter...,the mind behind the doped olympics the enhance...
1,None,'It feels right' - Whitlock to retire after Ol...,Gymnast Max Whitlock tells BBC Breakfast he wi...,https://www.bbc.co.uk/sport/av/gymnastics/6877...,'It feels right' - Whitlock to retire after Ol...,it feels right whitlock to retire after olymp...
2,BBC Sport,'Paris will be my final Olympics' - GB's Whitlock,Britain's most successful gymnast Max Whitlock...,https://www.bbc.co.uk/sport/gymnastics/68768442,Three-time gold medallist Max Whitlock says Pa...,paris will be my final olympics gb s whitlock...
3,https://www.facebook.com/bbcnews,Macron says Paris Olympics opening could be moved,The French president says the Olympics opening...,https://www.bbc.co.uk/news/world-europe-68815081,French President Emmanuel Macron says the Pari...,macron says paris olympics opening could be mo...
4,Matthew Henry,'Like an Olympic final' - Richards secures Par...,Current and former world champions Freya Colbe...,https://www.bbc.co.uk/sport/swimming/68736943,Current and former world champions Freya Colbe...,like an olympic final richards secures paris ...


APPLYING RECCOMENDATION ALGORITHM

In [178]:
# Initialize the HashingVectorizer
vectorizer = HashingVectorizer(n_features=1000, binary=True)

In [179]:

# Generate the matrix of hashed features
tfidf_matrix = vectorizer.fit_transform(metadata['text'])


In [180]:
d = tfidf_matrix.shape[1]
index = faiss.IndexFlatL2(d)


In [181]:

# Add the vectors to the index
index.add(tfidf_matrix.toarray())

In [182]:
metadata.sample(5)

,author,headline,abstract,links,content,text
15,JAMES ELLINGWORTH AP sports writer,Dozens of Russian athletes likely to qualify f...,Dozens of Russian athletes will likely be allo...,https://abcnews.go.com/Sports/wireStory/dozens...,"DUESSELDORF, Germany -- With 100 days to go un...",dozens of russian athletes likely to qualify f...
7,https://www.facebook.com/bbcnews,Watch: Olympic flame being lit in Greece's anc...,The lighting of the flame is the start of an i...,https://www.bbc.co.uk/news/live/world-68818723,A rehearsal was held on MondayImage caption: A...,watch olympic flame being lit in greece s anci...
28,Becky Sullivan,"World Athletics will pay $50,000 to Olympic go...","The prizes, which will also go to the winners ...",https://www.npr.org/2024/04/10/1243962306/olym...,Shaunae Miller-Uibo of the Bahamas wins the go...,world athletics will pay 50 000 to olympic gol...
49,Joseph Zucker,Report: Clippers' Kawhi Leonard to Get Final U...,Los Angeles Clippers star Kawhi Leonard will c...,https://bleacherreport.com/articles/10117143-r...,Jacob Kupferman/Getty Images\r\nLos Angeles Cl...,report clippers kawhi leonard to get final usa...
87,None,Alexis Sablone's Nike Dunks Look Too Cool (& P...,"For years, Alexis Sablone has blessed us with ...",https://www.highsnobiety.com/p/alexis-sablone-...,"For years, Alexis Sablone has blessed us with ...",alexis sablone s nike dunks look too cool preh...


In [183]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import pairwise_distances
import pandas as pd

def get_recommendations(query, vectorizer, index, metadata, k=10):
    # Transform the query into a vector
    query_vector = vectorizer.transform([query])
    
    def search(index, query_vector, k=10):
        D, I = index.search(query_vector, k)
        return I[0]

    # Use the index to find similar items
    neighbor_indices = search(index, query_vector.toarray(), k)
    
    # Fetch sentiment scores and labels corresponding to recommended headlines
    recommendations = []
    for idx in neighbor_indices:
        headline = metadata.iloc[idx]['headline']
        links = metadata.iloc[idx]['links']
        text = metadata.iloc[idx]['text']
        recommendations.append([headline, links, text])
    
    # Create a DataFrame from the recommendations
    recc_df = pd.DataFrame(recommendations, columns=['Headline', 'Links', 'text'])
    
    return recc_df

# Example usage
# x = input("Search: ")
query = search
recc_df = get_recommendations(query, vectorizer, index, metadata)
print("Recommendations for:", query)
print(recc_df)

# Save the recommendations DataFrame to a CSV file
# recc_df.to_csv('recommendations.csv', index=False)


Recommendations for: olympics
                                            Headline  \
0           Chinese doping scandal rocks Olympics...   
1  Olympic rings for the Paris Games will be disp...   
2        20 Colorful Slang Terms From Olympic Sports   
3  Paris 2024 torch lit in Olympics birthplace, r...   
4  Morgan breaks British record to qualify for Pa...   
5  Dozens of Russian athletes likely to qualify f...   
6     Morgan breaks British record to reach Olympics   
7  Nike is betting on a Summer Olympics sales tur...   
8            The Masters is something special - Rose   
9  GB's Lee to miss Paris Olympics after spinal s...   

                                               Links  \
0  https://consent.yahoo.com/v2/collectConsent?se...   
1  https://abcnews.go.com/Sports/wireStory/olympi...   
2  https://www.mentalfloss.com/posts/olympic-spor...   
3  https://www.aljazeera.com/sports/2024/4/16/par...   
4   https://www.bbc.co.uk/sport/av/swimming/68731115   
5  https://abcnew

In [184]:
recc_df.head()

,Headline,Links,text
0,Chinese doping scandal rocks Olympics...,https://consent.yahoo.com/v2/collectConsent?se...,chinese doping scandal rocks olympics chinese ...
1,Olympic rings for the Paris Games will be disp...,https://abcnews.go.com/Sports/wireStory/olympi...,olympic rings for the paris games will be disp...
2,20 Colorful Slang Terms From Olympic Sports,https://www.mentalfloss.com/posts/olympic-spor...,20 colorful slang terms from olympic sports he...
3,"Paris 2024 torch lit in Olympics birthplace, r...",https://www.aljazeera.com/sports/2024/4/16/par...,paris 2024 torch lit in olympics birthplace re...
4,Morgan breaks British record to qualify for Pa...,https://www.bbc.co.uk/sport/av/swimming/68731115,morgan breaks british record to qualify for pa...


SENTIMENT ANALYSIS

In [185]:
class NewsRecommendationSystem:
    def __init__(self, df):
        self.df = df

    def analyze_sentiment(self):
        sid = SentimentIntensityAnalyzer()
        nan_indices = self.df['text'].isna()
        # Replace NaN values with an empty string to avoid errors during sentiment analysis
        self.df.loc[nan_indices, 'text'] = ''

        self.df['sentiment_score'] = self.df['text'].apply(lambda x: sid.polarity_scores(x)['compound'])

    def recommend_news(self, threshold=0.5):
        if 'sentiment_score' not in self.df.columns:
            self.analyze_sentiment()
        return self.df[self.df['sentiment_score'] >= threshold]

In [186]:
if __name__ == "__main__":
    recommendation_system = NewsRecommendationSystem(recc_df)
    recommended_news = recommendation_system.recommend_news(threshold=0.1)

    print("Recommended news:")
    print(recommended_news)
    
def map_sentiment(score):
    if score > 0.3:
        return 'positive'
    elif 0.3 >= score >= -0.3:
        return 'neutral'
    else:
        return 'negative'

recc_df['sentiment_label'] = recc_df['sentiment_score'].apply(map_sentiment)
recc_df.drop(columns='text')

Recommended news:
                                            Headline  \
2        20 Colorful Slang Terms From Olympic Sports   
4  Morgan breaks British record to qualify for Pa...   
6     Morgan breaks British record to reach Olympics   
7  Nike is betting on a Summer Olympics sales tur...   
8            The Masters is something special - Rose   
9  GB's Lee to miss Paris Olympics after spinal s...   

                                               Links  \
2  https://www.mentalfloss.com/posts/olympic-spor...   
4   https://www.bbc.co.uk/sport/av/swimming/68731115   
6      https://www.bbc.co.uk/sport/swimming/68722374   
7  https://qz.com/nike-olympics-sales-sports-mark...   
8       https://www.bbc.co.uk/sport/av/golf/68743085   
9        https://www.bbc.co.uk/sport/diving/68674316   

                                                text  sentiment_score  
2  20 colorful slang terms from olympic sports he...           0.5574  
4  morgan breaks british record to qualify for pa...

,Headline,Links,sentiment_score,sentiment_label
0,Chinese doping scandal rocks Olympics...,https://consent.yahoo.com/v2/collectConsent?se...,-0.7003,negative
1,Olympic rings for the Paris Games will be disp...,https://abcnews.go.com/Sports/wireStory/olympi...,0.0000,neutral
2,20 Colorful Slang Terms From Olympic Sports,https://www.mentalfloss.com/posts/olympic-spor...,0.5574,positive
3,"Paris 2024 torch lit in Olympics birthplace, r...",https://www.aljazeera.com/sports/2024/4/16/par...,0.0000,neutral
4,Morgan breaks British record to qualify for Pa...,https://www.bbc.co.uk/sport/av/swimming/68731115,0.5574,positive
5,Dozens of Russian athletes likely to qualify f...,https://abcnews.go.com/Sports/wireStory/dozens...,0.0000,neutral
6,Morgan breaks British record to reach Olympics,https://www.bbc.co.uk/sport/swimming/68722374,0.5719,positive
7,Nike is betting on a Summer Olympics sales tur...,https://qz.com/nike-olympics-sales-sports-mark...,0.4019,positive
8,The Masters is something special - Rose,https://www.bbc.co.uk/sport/av/golf/68743085,0.7269,positive
9,GB's Lee to miss Paris Olympics after spinal s...,https://www.bbc.co.uk/sport/diving/68674316,0.4019,positive


FINAL RECOMENDATIONS

In [187]:
recc_df.head()

,Headline,Links,text,sentiment_score,sentiment_label
0,Chinese doping scandal rocks Olympics...,https://consent.yahoo.com/v2/collectConsent?se...,chinese doping scandal rocks olympics chinese ...,-0.7003,negative
1,Olympic rings for the Paris Games will be disp...,https://abcnews.go.com/Sports/wireStory/olympi...,olympic rings for the paris games will be disp...,0.0000,neutral
2,20 Colorful Slang Terms From Olympic Sports,https://www.mentalfloss.com/posts/olympic-spor...,20 colorful slang terms from olympic sports he...,0.5574,positive
3,"Paris 2024 torch lit in Olympics birthplace, r...",https://www.aljazeera.com/sports/2024/4/16/par...,paris 2024 torch lit in olympics birthplace re...,0.0000,neutral
4,Morgan breaks British record to qualify for Pa...,https://www.bbc.co.uk/sport/av/swimming/68731115,morgan breaks british record to qualify for pa...,0.5574,positive
